<a href="https://colab.research.google.com/github/umang-sh/tensorFlowMLSolutions/blob/master/Copy_of_logitReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import math

import tensorflow as tf
import pandas as pd
import numpy as np
from IPython import display
from sklearn import metrics
from matplotlib import gridspec
from matplotlib import pyplot as plt
from matplotlib import cm
from tensorflow.python.data import Dataset


tf.logging.set_verbosity(tf.logging.INFO)
pd.options.display.max_rows =10
pd.options.display.float_format ='{:.1f}'.format

california_housing_df=pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv",sep=",")

california_housing_df=california_housing_df.reindex(np.random.permutation(california_housing_df.index))

                      

In [0]:
def preprocess_features(california_housing_df):
    """Prepares input features from California housing data set.

  Args:
    california_housing_dataframe: A Pandas DataFrame expected to contain data
      from the California housing data set.
  Returns:
    A DataFrame that contains the features to be used for the model, including
    synthetic features.
  """
    selected_features=california_housing_df[
    ["latitude",
     "longitude",
     "housing_median_age",
     "total_rooms",
     "total_bedrooms",
     "population",
     "households",
     "median_income"]]
    
    display.display (selected_features.describe())
    processed_features=selected_features.copy()
    #creating a synthethic feature
    processed_features["rooms_per_person"]=california_housing_df["total_bedrooms"]/california_housing_df["population"]
    return processed_features
  
def preprocess_targets(california_housing_df):
    """Prepares target features (i.e., labels) from California housing data set.

    Args:
    california_housing_dataframe: A Pandas DataFrame expected to contain data
    from the California housing data set.
    Returns:
    A DataFrame that contains the target feature.
    """
    output_targets=pd.DataFrame()
    # Create a boolean categorical feature representing whether the
    # median_house_value is above a set threshold.
    output_targets["median_housing_value_is_high"]=(
    california_housing_df["median_house_value"]>265000).astype(float)
    display.display(output_targets.describe())
    return output_targets

In [0]:
#choosing first 12000 examples for training
training_examples= preprocess_features(california_housing_df.head(12000))
training_targets= preprocess_targets(california_housing_df.head(12000))

validation_examples=preprocess_features(california_housing_df.head(5000))
validation_targets=preprocess_targets(california_housing_df.head(5000))

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
count,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0
mean,35.6,-119.6,28.6,2658.8,541.9,1435.6,503.3,3.9
std,2.1,2.0,12.6,2214.9,425.6,1156.1,387.6,1.9
min,32.5,-124.3,1.0,8.0,1.0,8.0,1.0,0.5
25%,33.9,-121.8,18.0,1469.0,298.0,793.0,282.0,2.6
50%,34.2,-118.5,29.0,2134.0,436.0,1171.0,411.0,3.6
75%,37.7,-118.0,37.0,3161.2,649.0,1722.0,607.0,4.8
max,42.0,-114.3,52.0,37937.0,5471.0,35682.0,5189.0,15.0


,median_housing_value_is_high
count,12000.0
mean,0.3
std,0.4
min,0.0
25%,0.0
50%,0.0
75%,1.0
max,1.0


,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
count,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0
mean,35.6,-119.6,28.8,2645.7,537.7,1433.2,498.4,3.9
std,2.1,2.0,12.6,2207.9,423.1,1208.1,384.4,1.9
min,32.5,-124.3,1.0,11.0,3.0,13.0,5.0,0.5
25%,33.9,-121.8,18.0,1473.0,298.0,792.8,281.0,2.6
50%,34.3,-118.5,29.0,2124.5,432.5,1167.0,408.0,3.6
75%,37.7,-118.0,37.0,3122.0,639.0,1710.0,596.0,4.8
max,41.9,-114.3,52.0,37937.0,5471.0,35682.0,5189.0,15.0


,median_housing_value_is_high
count,5000.0
mean,0.2
std,0.4
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,1.0


In [0]:
def construct_feature_columns(input_features):
  
  """Construct the TensorFlow Feature Columns.

  Args:
    input_features: The names of the numerical input features to use.
  Returns:
    A set of feature columns
  """
  return set([tf.feature_column.numeric_column(my_feature) for my_feature in input_features ])

In [0]:
def my_input_fn(features,targets,batch_size=1,shuffle=true,num_epochs=None):
  """Trains a linear regression model.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
    
    #convert features into a dict of np arrays
    features={key:np.array(value) for key,value in dict(features).items()}
    print (features)
    
    #Use DataSet API to construct a dataset
    ds=DataSet.from_tensor_slices((features,targets))
    ds=ds.batch(batch_size).repeat(num_epochs)
    
    
  